In [1]:
path2train = "/home/e209440/data/train"
path2test = "/home/e209440/data/test"

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch as tc
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import transforms, utils

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

In [3]:
import os, glob, math,time
from scipy import misc



In [4]:
img_size = 50
sz = (img_size, img_size)


In [5]:
def process_image(img_file):
    img = misc.imread(img_file)
    img = misc.imresize(img, sz).transpose((2,0,1)).astype('float32') / 255.0
    return img

In [6]:
def load_traindata():

    start = time.time()
    X_train = []
    Y_train = []

    for j in range(1):
        print('Load folder c{}'.format(j))
        path = os.path.join(path2train, 'c' + str(j), '*.jpg')
        files = glob.glob(path)
        X_train.extend(process_image(im_file) for im_file in files)
        Y_train.extend([j]*len(files))

    end = time.time() - start
    print("Time: %.2f seconds" % end)
    return (X_train,Y_train)

In [21]:
X_train, y_train = load_traindata()

Load folder c0
Time: 24.09 seconds


In [8]:
len(X_train)

2489

In [10]:
X_train[0].shape

(3, 50, 50)

In [9]:
len(y_train)

2489

In [ ]:
def process_test_image(img_file):
    return process_image(img_file), os.path.basename(img_file)

In [29]:
def load_testdata():
    start = time.time()
    X_test    = []
    X_test_id = []

    path  = os.path.join(path2test, '*.jpg')
    files = glob.glob(path)

#    results = (process_test_image(im_file) for im_file in files)  # Original

#   Ajuste para pegar poucas imagens : 

    idx = 0
    for im_file in files:
        idx += 1
        if idx == 20:
            break
        X_test.extend(process_image(im_file))    
#        results = process_image(im_file) # Lembrar de discutir função process_test_image
            

#    X_test, X_test_id,_ = zip(*results)

    end = time.time() - start
    print("Time: %.2f seconds" % end)
    return (X_test)


In [30]:
X_test = load_testdata()

Time: 0.85 seconds


In [33]:
print(len(X_test))
X_test[2].shape

57


(50, 50)

In [ ]:
few_samples = True
if few_samples:
    n_samples = 100
else : 
    n_samples = data['X_train'].shape[0]

In [ ]:
X_train = data['X_train'][:n_samples]
y_train = data['y_train'][:n_samples]
X_test = data['X_test'][:n_samples]
y_test = data['y_test'][:n_samples]

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
class CRDataset(Dataset):
    
    def __init__(self, data, target, transform):
        assert data.shape[0] == target.shape[0]
        
        self.data = data
        self.target = torch.LongTensor(target.astype(int))
        self.transform = transform
    
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self, i):
        data = self.transform(self.data[i, :, :, :])
        return (data, self.target[i])

